# List of all available endpoints

#### The notebook initialization

In [1]:
from IPython.display import HTML
from init import *

## Endpoint: /

In [2]:
HTML(requests.get(server).text)

## Endpoint: /version

In [3]:
requests.get(f'{server}/version').json().split('\n')

['mc-0.1.1 built=[Mar 15 2023 15:31:08] #deps=2',
 '  KernelModelArgs-0.5.0 #deps=0',
 '  Model-0.5.0 #deps=1',
 '    Parameter-0.2.0 #deps=0',
 '']

## Prometheus metrics

In [4]:
requests.get(f'{server}/metrics').text.split('\n')

['# TYPE get_functions counter',
 'mc_get_functions 2',
 '# TYPE get_metrics counter',
 'mc_get_metrics 3',
 '# TYPE get_version counter',
 'mc_get_version 2',
 '# TYPE model counter',
 'mc_model 4',
 '# TYPE model_version5 counter',
 'mc_model_version5 3',
 '# TYPE mc_get_functions summary',
 'mc_get_functions_count 2',
 'mc_get_functions_seconds_sum 0.12049199999999999',
 'mc_get_functions_seconds_mean 0.060246',
 'mc_get_functions_seconds_stddev 0.060113',
 '# TYPE mc_get_metrics summary',
 'mc_get_metrics_count 2',
 'mc_get_metrics_seconds_sum 0.000274',
 'mc_get_metrics_seconds_mean 0.000137',
 'mc_get_metrics_seconds_stddev 6e-06',
 '# TYPE mc_get_version summary',
 'mc_get_version_count 2',
 'mc_get_version_seconds_sum 4.6e-05',
 'mc_get_version_seconds_mean 2.3e-05',
 'mc_get_version_seconds_stddev 8e-06',
 '# TYPE mc_model summary',
 'mc_model_count 4',
 'mc_model_seconds_sum 0.15934399999999999',
 'mc_model_seconds_mean 0.039836',
 'mc_model_seconds_stddev 0.0626546',
 '# TYP

## Model

In [5]:
model = {'TimeStart': 0,
 'TimeSteps': 1000,
 'NumPaths': 100000,
 'updaters': [
     {'name': 'IndependentGaussian'},
     {'name': 'BrownianMotion', 'start': 0.1, 'args': [0.2,2]}
 ],
 'evaluations': [{'state': 0, 'time': 2}]
}
model_json = json.dumps(model)
model_json

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2}]}'

In [6]:
requests.post(f'{server}/model',model_json).json()

{'histograms': [],
 'mean': [0.5074735283851624],
 'names': ['BrownianMotion'],
 'npaths': [100000],
 'run_time_seconds': {'mean': 0.22039972245693207,
  'n': 512,
  'skewness': None,
  'stddev': 0.00039053853834047914},
 'skewness': [-4.932541196467355e-05],
 'stddev': [2.8231453895568848],
 'time_points': [2.0],
 'time_steps': [1000]}

## Endpoint: "/functions"

In [7]:
functions = requests.get(f'{server}/functions').json()
functions

[{'doc_md': "\n# Barrier\n\nA barrier state starts from some initial value S0.\nOn each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will\nbe set to STATE=valueOnCrossing.\n\nFor direction 'up':\n\n    if(Sold<level and Snew>=level)\n        barrierState = valueOnCrossing;\n\nFor direction 'down':\n\n    if(Sold>level and Snew<=level)\n        barrierState = valueOnCrossing;\n\n## References\n\nThe barrier function expectes exactly one reference: an **underlying**.\n\n### [ref 0]\n\nThe underlying process.\n\n## Arguments\n\n### [arg 0] level\nBarrier level\n\n### [arg 1] valueOnCrossing\nThe value we set when the barrier is crossed.\n\n### [arg 2] direction\n\nThis is the barrier crossing condition.\n\n| value | direction  |\n| ------------- |-------------:|\n| -1 | down          |\n|  0 | any (up or down) |\n| +1 | up |\n    .has_start  = false,\n    .nargs_min  = 4,\n    .nrefs_min  = 1,\nr of time windows\n\nThe number of time

In [8]:
from IPython.display import Markdown as md
doc = ''
for f in functions:
    doc += f"# {f['name']}\n"
    doc += f"## {f['title']}\n"
    doc += f['doc_md'] + '\n\n'
    doc += '---\n\n'
md(doc)

# Barrier
## Barrier (up/down) with windows

# Barrier

A barrier state starts from some initial value S0.
On each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will
be set to STATE=valueOnCrossing.

For direction 'up':

    if(Sold<level and Snew>=level)
        barrierState = valueOnCrossing;

For direction 'down':

    if(Sold>level and Snew<=level)
        barrierState = valueOnCrossing;

## References

The barrier function expectes exactly one reference: an **underlying**.

### [ref 0]

The underlying process.

## Arguments

### [arg 0] level
Barrier level

### [arg 1] valueOnCrossing
The value we set when the barrier is crossed.

### [arg 2] direction

This is the barrier crossing condition.

| value | direction  |
| ------------- |-------------:|
| -1 | down          |
|  0 | any (up or down) |
| +1 | up |
    .has_start  = false,
    .nargs_min  = 4,
    .nrefs_min  = 1,
r of time windows

The number of time windows (say **m**) when the barrier will be checked.
The next **m * 2** parameters will be treated as [from,to) pairs, time windows
where the barrier condition will be checked. If **m==0**, the barrier condition
will be checked on all time step.

## Examples

### S0=0, args=[1000,1,-1,0,0]

The barrier inital state is 0. We check on any time step (because arg[4]==0)
the the barrier level==1000 will be crossed in the down direction (for example,
from 1000.5 to 1000). If the condition is NOT fulfilled, we don't change the
barrier state. Otherwise, we set the barrier value (arg[3]==0 means 'set')
to value 1 (because arg[1]==1). Obvioulsy, only the first crossing over
the barrier will have an effect, when we change the barrier state from 0
(because S0==0) to 1 (because arg[1]==1 and arg[3]==0)

### S0=0, args=[1000,1,-1,0,2,111.1,222.2,333.3,444.4]

The same as above, but wee only check the barrier condition if the time
is in one of the two (because arg[5]==2) windows:

- [111.1,222.2)
- [333.3,444.4)

### S0=0, args=[1000,1,-1,1,2,111.1,222.2,333.3,444.4]

The same as above, we 'count' (arg[3]==1 is the increment)
how many times the barrier is crossed in the 'down' direction.


---

# BrownianMotion
## BrownianMotion process which dependes on drfit and diffusion.

# BrownianMotion
The function expects either 2 arguments (drift,diffusion) OR 2 references to (drift,diffusion) processes.
    

---

# CorrelatedGaussian
## Generate correlated normal random variables.

# CorrelatedGaussian

## Initial state

Not applicable, the function has no initial state.

## Arguments

The function has exactly arguments. It is a correlation parameter between the two states passed in the refs arguments.

|   | name | description | range |
|---|------|-------------|-------|
| 0 | rho  | correlation  parameter between ref[0] and ref[1] states | [-1,1] |

## References

|   | name | description |
|---|------|-------------|
| 0 | src  | sourcerandom variable number 
| 1 | dst  | destination random variable number 

A random variable will be read from the src state, and a new random variable will
be generated using src and correlation parameter rho. The result will be put into
dst random variable state. src random variable will not be changed, dst random variable
will be overwritten.

## Example:

- start = Not applicable
- args = [-0.77]
- refs = [2,5]

Read random variable from state 2, generate a new gaussian random variable with
correlation -0.77 and save the result into random variable state 5.



---

# Division
## Division

# Division

Compute division of two processes.

If abs(denominator)<=eps, than the division result will be not-a-number.

## Initial state

Not used, the function will override previously computed values.

## Arguments

The function has exactly one arguments. It is a non-negative number eps. 
If denominator state absolute value is less than eps, the division result will be set to NaN.
This can be used to avoid producing +/- infinity values for the division result.
A value 0 can be used (which essentially will allow division by zero).

|   | name | description | range |
|---|------|-------------|-------|
| 0 | eps  | abs(denominator) min value | non-negative |

## References

|   | name |
|---|------|
| 0 | Numerator |
| 1 | Denominator |

## Example:

- start = Ignore
- args = [0.01]
- refs = [2,5]

Take a state[5] value, if abs(state[5])<=0.01 set the division result to NaN.
Otherwise, computes state[2]/state[5] and saves the result into current state.



---

# GeometricalBrownianMotion
## GeometricalBrownianMotion process which dependes on drfit and diffusion functions.

# BrownianMotion
The function expects either 2 arguments (drift,diffusion) OR 2 references to (drift,diffusion) processes.
    

---

# HullWhiteModel
## HullWhiteModel


---

# IndependentGaussian
## Generate independent normal-distributed random variables

# IndependentGaussian

## Initial state

Not applicable, the function has no initial state.

## Arguments

The function has no arguments. It will generate normally distributed random variables with
mean=0 and volatility=1.

## References

If the number of references is 0, the function will calculate random variables
for all states of the system.

Otherwise (if the number of references is >0) only the states mentioned in the refs
will be populated.

## Examples:

### Example 1

- start = Not applicable
- args = []
- refs = []

The function will compute random variables only for all states of the system.

### Example 2

- start = Not applicable
- args = []
- refs = [0,3]

The function will compute random variables only for states 0 and 3.



---

# Linear1DInterpolation
## One dimensional function interpolation.
# Linear1DInterpolation

Perform one demensional interpolation between [Xmin,Xmax].
The number of interpolation points
is deduced from the number of arguments passed to the function.
The first two arguments are Xmin,Xmax. The rest arguments are Y-values on the
regular grid between [Xmin,Xmax]. Thus, the total nummber of arguments
must be at least 4. And the total number of interpolation points is narg-2.

### Example: args=[1,2,3,4,5]
there are three interpolation points (5-2=3):

- Xmin = 1.0
- Xmax = 2.0
- Y(X=1.0) = 3
- Y(X=1.5) = 4
- Y(X=2.0) = 5

The interpolation function argument is passed as 'Xref'. If Xref=-1, then the argument is time.
Otherwise, it is a state number to will be used as an 'x'-argument.

Linear1DInterpolation returns Y(Xmin) if X<Xmin and Y(Xmax) if X>Xmax.

Linear1DInterpolation does not use the old state value.
Thus, the starting process state value is ignored.



---

# Multiplication
## Multiplication

# Multiplication

The function computes arg[0] * state[ref[0]] * state[ref[1]] * ...

## Initial state

Ignored, the function has state (and computes it), but the old value (computed on the previous step)
is not used.

## Arguments

The function has one arguments.

|   | name |
|---|------|
| 0 | multiplication factor |

## References

Non-zero number of references: nrefs>0. All states will be multipplied.

## Example:

- start = ignored
- args = [0.5]
- refs = [0,7,9]

Computes 0.5*state[0]*state[7]*state[9]



---

# Option
## Compute call/put
# Option (finance)

## Initial state

The updater has no initial state.

## Arguments

|   | name   | description   |
|---|--------|---------------|
| 0 | strike |               |
| 1 | type   | 0-call, 1-put |

## References

|   | name   | description   |
|---|--------|---------------|
| 0 | underlying | underlying process S |
    

---

# Polynom
## n-th order polynom.

# Polynom

The polynom order is taken from the number of passed arguments.
Minimum number is 1, which will be treated as constant function.
Otherwise it will compute a0+a1*x+a2*x*x+..., where 'x'-is the
polynom argumnt (see below) and a0,a1... are arguments of the
function.

The polynopm argument 'x' is passed as 'Xref'. If Xref=-1, then the argument is time.
Otherwise, it is a state number to which will be used as an 'x'-argument.



---

# SABR_F
## SABR_F


---

# SABR_S
## SABR_S


---

# ZeroCouponBond
## Zero-coupon bond (dsiscount factor)


---



In [14]:
import mcsdk as sdk
for j in functions:
    f = sdk.UpdaterDoc()
    sdk.UpdaterDoc_to_json(j,f)